In [2]:
import numpy as np
from scipy.spatial import distance
import glob
import cv2
import sys
import seaborn as sns
from abc import ABC, abstractmethod, abstractproperty
import matplotlib.pyplot as plt
plt.rcParams["font.family"] = "Arial"

In [3]:
import random
import math
from IPython.display import clear_output, display
from IPython.display import HTML as html
import time
import io
import matplotlib.patches
from calysto.graphics import *
import threading
from ipywidgets import *
display(HTML('''<style>
            .widget-label { min-width: 20ex !important; }
            .widget-text { width: 30ex; }
            </style>'''))
# HTML('<style> .widget-text { width: auto; } </style>')

%matplotlib inline
sns.set(style="ticks")

HTML(value='<style>\n            .widget-label { min-width: 20ex !important; }\n            .widget-text { wid…

In [52]:
RANDOM_SEED = 0

## Firefly classes

In [4]:
class Firefly(ABC):
    
    #-------- Abstract properties --------#
    def __init__(self, sex, position_x, position_y):
        self.sex = sex                 # String: f or m
        self.position_x = position_x   
        self.position_y = position_y
    
    #-------- Abstract methods --------#
    def __repr__(self):
        return (f"sex: {self.sex} -- xy: {self.position_x, self.position_y}")
    

In [54]:
np.random.seed(RANDOM_SEED)

class MaleFirefly(Firefly):
    def __init__(self, sex, position_x, position_y, 
                 id_number, num_total_steps, step_size, 
                 flash_interval, turning_angle_distribution):
        super().__init__(sex, position_x, position_y)
        self.id_number = id_number
        self.num_total_steps = num_total_steps
        self.step_size = step_size
        self.flash_interval = flash_interval 
        self.turning_angle_distribution = turning_angle_distribution
        
        # Attributed initialized and used inside this class
        self.turning_angle = np.pi * (np.random.rand()-0.5)              # Initial random turning angle in radians
        self.turning_angle_history = [self.turning_angle]                # Keep track of its turning angles over time 
        
        self.trajectory_history = [(self.position_x, self.position_y)]   # Keep track of its xy (tuple) position over time
        
        self.flash_counter = 1                      # Counts the time that has passed since last flash
        self.flash = False                          # Flag for whether the firefly is flashing at a given time
        self.flash_history = [0]                    # Keep track of its flashing history over time (list of booleans)
     
    def pick_turning_angle(self):
        ''' For each step, a male firefly picks a random turning angle from
        the range using the previous turning angle. '''
        self.turning_angle += self.turning_angle_distribution * 2 * (np.random.rand()-0.5)
        self.turning_angle_history.append(self.turning_angle)

    def check_flash(self):
        ''' Based on the flash interval, if self.flash_counter == self.flash_interval,
        the firefly would flash again. '''
        if self.flash_counter == self.flash_interval:
            self.flash = True
            
    def check_arena_boundary(self, initial_arena_size, dim):
        return dim <= -initial_arena_size or dim >= initial_arena_size 
    
    def take_step(self, initial_arena_size):
        ''' After picking a random turning angle, the firefly takes a step,
        updating its xy position and flash counter. This process represents
        the correlated random walk model for male firefly motion. '''
        
#         next_x = self.position_x + self.step_size * np.cos(self.turning_angle)
#         next_y = self.position_x + self.step_size * np.sin(self.turning_angle)
        
#         move_in_x = self.check_arena_boundary(initial_arena_size, next_x)
#         move_in_y = self.check_arena_boundary(initial_arena_size, next_y)
        
#         if move_in_x:
#             self.position_x += next_x
#         else:
#             self.position_x = self.position_x
        
#         if move_in_y:
#             self.position_y += next_y 
#         else:
#             self.position_y = self.position_y 
        
        self.position_x += self.step_size * np.cos(self.turning_angle)
        self.position_y += self.step_size * np.sin(self.turning_angle)
           
#         if self.check_arena_boundary(initial_arena_size, self.position_x):
#             self.position_x -= self.step_size*3 * np.cos(self.turning_angle)
# #             self.position_x += self.step_size*3 * np.cos(-self.turning_angle)
            
#         if self.check_arena_boundary(initial_arena_size, self.position_y):
#             self.position_y -= self.step_size*3 * np.sin(self.turning_angle)
#             self.position_y += self.step_size*3 * np.sin(-self.turning_angle)
            
#         rechoose_x = True
#         while rechoose_x:
#             rechoose_x = self.check_arena_boundary(initial_arena_size, self.position_x)
#             # Rechoose position to move
# #             self.position_x += self.step_size * np.cos(self.turning_angle)
#             self.position_x -= self.step_size * np.cos(self.turning_angle)
            
            
#         rechoose_y = True
#         while rechoose_y:
#             rechoose_y = self.check_arena_boundary(initial_arena_size, self.position_y)
#             # Rechoose position to move
# #             self.position_y += self.step_size *  np.sin(self.turning_angle)
#             self.position_y -= self.step_size *  np.sin(self.turning_angle)
            
            
            
    
        self.trajectory_history.append((self.position_x, self.position_y))
                
        if self.flash:
            self.flash_history.append(1)
            self.flash = False            # Reset flash
            self.flash_counter = 1        # Reset flash counter
        else: 
            self.flash_history.append(0)
            self.flash_counter += 1       # Increment flash counter
            
    def record_history(self):
        male_dict = {
            "id_number"             : self.id_number,
            "turning_angle_history" : np.array(self.turning_angle_history),
            "trajectory_history"    : np.array(self.trajectory_history),
            "flash_history"         : np.array(self.flash_history)
        }
        return male_dict        

    def __repr__(self):
        return (f"sex: {self.sex} -- id: {self.id_number} -- xy: {self.position_x:0.2f}, {self.position_y:0.2f} -- angle: {self.turning_angle:0.2f} -- flash interval: {self.flash_interval} -- flashing: {self.flash}")

In [55]:
np.random.seed(RANDOM_SEED)

class FireflyFactory():
    ''' Generate male firefly objects, each with a unique id number, random initial xy, and flash interval. '''
    
    def create_male_firefly(sex, position_x, position_y, 
                            id_number, num_total_steps, step_size, 
                            flash_interval, turning_angle_distribution):
        
        firefly = MaleFirefly(sex, position_x, position_y, id_number, num_total_steps, 
                              step_size, flash_interval, turning_angle_distribution)
        return firefly

In [56]:
np.random.seed(RANDOM_SEED)

class FireflyCollection():
    ''' Collection contains all male fireflies created by the FireflyFactory, with default model parameters
    as arguments. '''
    
    def __init__(self, num_males=20, step_size=1.0, num_total_steps=1000, 
                 flash_interval_min=20, flash_interval_max=50, turning_angle_distribution=np.pi/10,
                 initial_arena_size=1000):
        
        self.num_males = num_males
        self.step_size = step_size
        self.num_total_steps = num_total_steps
        self.flash_interval_min = flash_interval_min
        self.flash_interval_max = flash_interval_max
        self.turning_angle_distribution = turning_angle_distribution
        self.initial_arena_size = initial_arena_size
        
        # List to contain all males firefly objects
        self.male_fireflies = []
        self.generate_fireflies(FireflyFactory)
        
    def generate_fireflies(self, FireflyFactory):
        for firefly_i in range(self.num_males):
            # Generate random parameter values to make varied male fireflies
            sex = "m"
            id_number = firefly_i
            position_x = self.initial_arena_size*2 * (np.random.rand() - 0.5)
            position_y = self.initial_arena_size*2 * (np.random.rand() - 0.5)
            num_total_steps = self.num_total_steps
            step_size = self.step_size
            flash_interval = np.random.randint(self.flash_interval_min, self.flash_interval_max)
            turning_angle_distribution = self.turning_angle_distribution
            
            firefly = FireflyFactory.create_male_firefly(sex, position_x, position_y, 
                                                         id_number, num_total_steps, step_size, 
                                                         flash_interval, turning_angle_distribution)
            self.male_fireflies.append(firefly)
            
    def __iter__(self):
        for ff in self.male_fireflies:
            yield ff

## World class

In [36]:
# # Model Parameters
# num_males = 30                        # Number of fireflies in arena 
# step_size = 1.0                       # Velocity aka step size
# num_total_steps = 100                # Number of steps per trajectory
# turning_angle_distribution = np.pi/20 # Width of random walk turning angle distribution. Lower: straighter.
# flash_interval_min = 0                # Number of steps bw flashes, random value bw min & max for each male
# flash_interval_max = 10
# initial_arena_size = num_total_steps  # Arena size to start with

In [57]:
np.random.seed(RANDOM_SEED)

class World:
    ''' Represent the entire model with the 2D space and all the fireflies and 
    their interactions '''
    
    def __init__(self, num_males=30, step_size=1.0, num_total_steps=50, turning_angle_distribution=np.pi/8,
                 flash_interval_min=5, flash_interval_max=50, initial_arena_size=100, track_on=True):
        
        self.num_males = num_males       # Constant number of fireflies in sky
        self.step_size = step_size
        self.num_total_steps = num_total_steps
        self.turning_angle_distribution = turning_angle_distribution
        self.flash_interval_min = flash_interval_min
        self.flash_interval_max = flash_interval_max
        self.initial_arena_size = initial_arena_size
        
        # For plotting - option of showing track or not
        self.track_on = track_on
        
        self.state = "stopped"            # State of the whole world
        self.setup()                      # When World is instantiated, setup a world aka dark sky with fireflies
        
    def setup(self):
        ''' Set up the world with Firefly objects instantiated at their initial positions. '''
        self.ticks = 0           # Simulation time, aka num_total_steps
        
        self.all_fireflies_history = {}   # Keep history of all positions and flashes over time
        self.num_flashes_history = [0]    # Keep history of number of flashes over time
        self.distance_bw_flashes = [0]    # Keep history of the distance between fireflies that flash over time
        
        # List of Firefly objects, created using FireflyCollection and FireflyFactory
        self.firefly_collection = FireflyCollection(self.num_males, self.step_size, self.num_total_steps, 
                                     self.flash_interval_min, self.flash_interval_max, 
                                     self.turning_angle_distribution,
                                     self.initial_arena_size)

    def get_status(self, num_flashes, avg_distance_bw_flashes):
        self.num_flashes_history.append(num_flashes)
        
        self.distance_bw_flashes.append(avg_distance_bw_flashes)
        
    def update_fireflies(self):
        num_flashes = 0
        flash_positions = []
        
        for ff_i, ff in enumerate(self.firefly_collection):
            ff.pick_turning_angle()
            ff.check_flash()
            if ff.flash:
                num_flashes += 1
                flash_positions.append((ff.position_x, ff.position_y))
            ff.take_step(self.initial_arena_size)
            hist_dict = ff.record_history() 
            self.all_fireflies_history[f"firefly_{ff_i}"] = hist_dict

        if flash_positions:
            avg_distance_bw_flashes = np.mean(distance.cdist(flash_positions, flash_positions, 'euclidean'))
        else:
            avg_distance_bw_flashes = 0
        
        self.get_status(num_flashes, avg_distance_bw_flashes)

    def run2(self):
        self.state = "running"
        for t_i in range(self.num_total_steps):
            self.ticks += 1
            self.update_fireflies()
        
            sys.stdout.write(f"\rTimestep {t_i+1}/{self.num_total_steps}")
            sys.stdout.flush()
            
        self.state = "stopped"       
        
    def run(self, widgets):
        self.state = "running"
        for t_i in range(self.num_total_steps):
            self.ticks += 1
            self.update_fireflies()

            widgets["plot_data"].value = self.plot_data()
            widgets["plot_simulation"].value = self.plot_simulation(t_i)
            widgets["ticks"].value = str(self.ticks)
            
        self.state = "stopped"
    
    def plot_data(self, show=False):
        fig, ax = plt.subplots(2, 1, sharex=True)  # figsize=(5,4)
        
        # Plot number of flashing fireflies
        ax[0].plot(self.num_flashes_history, c="orange")
        ax[0].spines['right'].set_visible(False)
        ax[0].spines['top'].set_visible(False)
        ax[0].set_ylabel('Count')
        ax[0].set_title("Number of flashing fireflies")
        
        # Plot distance between flashers
        ax[1].plot(self.distance_bw_flashes, c="green")
        ax[1].spines['right'].set_visible(False)
        ax[1].spines['top'].set_visible(False)
        ax[1].set_ylabel('Distance')
        ax[1].set_title("Distance between flashes")
        
        ax = fig.add_subplot(111, frameon=False)
        ax.set_xticks([])
        ax.set_yticks([])
        ax.set_xlabel('Time', labelpad=25) 

        fig.subplots_adjust(wspace=0.25, hspace=0.5)
        
        if show:
            plt.show()
            
        bytes = io.BytesIO()
        plt.savefig(bytes, format='svg')
        svg = bytes.getvalue()
        plt.close(fig)
        return svg.decode()
    
    def plot_simulation(self, t_i, show=False):
        x_min = -self.initial_arena_size
        x_max = self.initial_arena_size
        y_min = -self.initial_arena_size
        y_max = self.initial_arena_size
        
        color = plt.cm.rainbow(np.linspace(0, 1, self.num_males))

        fig = plt.figure(1, figsize=(7, 7))  # figsize=(8, 7)
        ax = fig.add_subplot(1, 1, 1)

        # Sky with fireflies
        for ff_i, (key, val) in enumerate(self.all_fireflies_history.items()):
            
            ''' For a single firefly, extract its info and plot its trajectory. '''
            firefly_data = self.all_fireflies_history[key]

            x = firefly_data['trajectory_history'][:,0]
            y = firefly_data['trajectory_history'][:,1]
            flashes = np.where(firefly_data['flash_history'] == 1)
            
            x_flashes = x[flashes]
            y_flashes = y[flashes]
            
            # Plot position trajectory
            if self.track_on:
                plt.plot(x[0:t_i], y[0:t_i], lw=1, zorder=0, label=f"{ff_i+1}", c=color[ff_i])

            # Plot flashing patterns
            for global_i, f_i in enumerate(flashes[0]):
                if t_i == f_i:
                    plt.scatter(x_flashes[global_i], y_flashes[global_i], c='orange', s=40)

        # Set black background
        ax = plt.gca()
        ax.set_facecolor('xkcd:black')
        ax.xaxis.set_visible(False)
        ax.yaxis.set_visible(False)
        plt.xlim(x_min-50, x_max+50)
        plt.ylim(y_min-50, y_max+50)

        if show:
            plt.show()

        bytes = io.BytesIO()
        plt.savefig(bytes, format='svg', bbox_inches='tight', pad_inches=0)
        svg = bytes.getvalue()
        plt.close(fig)
        return svg.decode()


## Widget time

In [71]:
np.random.seed(RANDOM_SEED)

firefly_world = World()

layout = Layout(border='0px solid red', width='350px', height='30px',
               margin='0px 0px 0px 0px')

# Widgets
# Buttons, text
ticks = Text(description="Ticks:", layout=Layout(border='0px solid red', width='200px', height='30px', margin='0px 0px 0px 60px'))
setup_button = Button(description="setup/reset")
go_button = Button(description="go")
plot_data = HTML()
plot_simulation = HTML(layout=Layout(border='0px solid red', margin='0px 0px 0px 0px'))
num_males_slider = IntSlider(description="Number of fireflies:", value=30, min=1, max=100, layout=layout)
step_size_slider = IntSlider(description="Step size:", value=1, min=1, max=5, layout=layout)
num_steps_slider = IntSlider(description="Number of ticks:", value=50, min=10, max=300, layout=layout)
turning_angle_slider = FloatSlider(description="Turning angle width:", value=np.pi/8, min=np.pi/15, max=np.pi/5, layout=layout)
flash_min_slider = IntSlider(description="Flashing interval min:", value=5, min=1, max=15, layout=layout)
flash_max_slider = IntSlider(description="Flashing interval max:", value=30, min=10, max=100, layout=layout)
# Checkbox
trackon_checkbox = Checkbox(description="Show track", layout=Layout(border='0px solid red', width='500px', margin='0px 0 -35px 0px'))
##################################################################

# Color settings
setup_button.style.button_color = 'lightgreen'
go_button.style.button_color = 'lightgreen'
num_males_slider.style.handle_color = 'lightgreen'
step_size_slider.style.handle_color = 'lightgreen'
num_steps_slider.style.handle_color = 'lightgreen'
turning_angle_slider.style.handle_color = 'lightgreen'
flash_min_slider.style.handle_color = 'lightgreen'
flash_max_slider.style.handle_color = 'lightgreen'

##################################################################
# Layout
row1 = HBox([setup_button, go_button, trackon_checkbox])
row2 = HBox([num_males_slider, step_size_slider])
row3 = HBox([num_steps_slider, turning_angle_slider])
row4 = HBox([flash_min_slider, flash_max_slider])
row5 = HBox([plot_data], layout=Layout(border='solid 0px lightgreen', height="350px", width="500px"))
widgets = HBox([VBox([row1, row2, row3, row4, row5], layout=Layout(border='solid 0px black', width="600px")), 
                VBox([ticks, plot_simulation], layout=Layout(border='solid 0px black', width="500px"))])

##################################################################
# Update
ticks.value = str(firefly_world.ticks)
plot_data.value = str(firefly_world.plot_data())
plot_simulation.value = str(firefly_world.plot_simulation(int(ticks.value)))
num_males_slider.value = firefly_world.num_males
step_size_slider.value = firefly_world.firefly_collection.step_size
num_steps_slider.value = firefly_world.num_total_steps
turning_angle_slider.value = firefly_world.firefly_collection.turning_angle_distribution
flash_min_slider.value = firefly_world.firefly_collection.flash_interval_min
flash_max_slider.value = firefly_world.firefly_collection.flash_interval_max
trackon_checkbox.value = firefly_world.track_on

def update(args):
    if args["name"] != "value":
        return
    owner = args["owner"]
    name = args["name"]
    value = args["new"]
    
    if owner == num_males_slider:
        firefly_world.num_males = value
    elif owner == step_size_slider:
        firefly_world.step_size = value
    elif owner == num_steps_slider:
        firefly_world.num_total_steps = value
    elif owner == turning_angle_slider:
        firefly_world.turning_angle_distribution = value
    elif owner == flash_min_slider:
        firefly_world.flash_interval_min = value
    elif owner == flash_max_slider:
        firefly_world.flash_interval_max = value
    elif owner == trackon_checkbox:
        firefly_world.track_on = value
        
def setup(widgets):
    firefly_world.setup()
    plot_data.value = firefly_world.plot_data()
    plot_simulation.value = firefly_world.plot_simulation(int(ticks.value))
    ticks.value = str(firefly_world.ticks)
        
def run(widgets):
    if firefly_world.state == "stopped":
        w = {"plot_data": plot_data, "ticks": ticks, "plot_simulation": plot_simulation} 
        firefly_world.run(w)  

num_males_slider.observe(update)         
step_size_slider.observe(update)
num_steps_slider.observe(update)
turning_angle_slider.observe(update)
flash_min_slider.observe(update)
flash_max_slider.observe(update)
trackon_checkbox.observe(update)

setup_button.on_click(setup)
go_button.on_click(run)

widgets

## Main

In [222]:
# Simulate a sky of flashing and moving fireflies
firefly_collection = FireflyCollection(num_males, step_size, num_total_steps, 
                                     flash_interval_min, flash_interval_max, 
                                     turning_angle_distribution,
                                     initial_arena_size)
all_fireflies_history = {}
for t_i in range(num_total_steps):
    for ff_i, ff in enumerate(firefly_collection):
        ff.pick_turning_angle()
        ff.check_flash()
        ff.take_step()
        hist_dict = ff.record_history() 
        
        all_fireflies_history[f"firefly_{ff_i}"] = hist_dict

NameError: name 'num_males' is not defined

In [ ]:
# Get min, max x, y across all fireflies to set arena 
x_min = num_total_steps * 2
x_max = - num_total_steps * 2
y_min = num_total_steps * 2
y_max = - num_total_steps * 2

for d in all_fireflies_history.values():
    x_min_d = min(d['trajectory_history'][:,0])
    x_max_d = max(d['trajectory_history'][:,0])
    y_min_d = min(d['trajectory_history'][:,1])
    y_max_d = max(d['trajectory_history'][:,1])
    
    if x_min_d < x_min:
        x_min = x_min_d
    if x_max_d > x_max:
        x_max = x_max_d
    if y_min_d < y_min:
        y_min = y_min_d
    if y_max_d > y_max:
        y_max = y_max_d
    
x_min, x_max, y_min, y_max

## Generate movie

In [ ]:
!rm MovieFrames/*.png

color = plt.cm.rainbow(np.linspace(0, 1, num_males))

track_on = True

for frame_i in range(num_total_steps):
    sys.stdout.write(f"\rFrame {frame_i+1}/{num_total_steps}")
    sys.stdout.flush()
    
    fig = plt.figure(figsize=(8,7))
    ax = fig.add_subplot(1, 1, 1)

    for ff_i, (key, val) in enumerate(all_fireflies_history.items()):
        ''' For a single firefly, extract its info and plot its trajectory. '''
        firefly_data = all_fireflies_history[key]

        x = firefly_data['trajectory_history'][:,0]
        y = firefly_data['trajectory_history'][:,1]
        flashes = np.where(firefly_data['flash_history'] == 1)
        x_flashes = x[flashes]
        y_flashes = y[flashes]
        
        # Plot position trajectory
        if track_on:
            plt.plot(x[0:frame_i], y[0:frame_i], lw=1, zorder=0, label=f"{ff_i+1}", c=color[ff_i])

        # Plot flashing patterns
        for global_i, f_i in enumerate(flashes[0]):
            if frame_i == f_i:
                plt.scatter(x_flashes[global_i], y_flashes[global_i], c='orange', s=40)
                
#             else:
#                 plt.scatter(0, 0, c="gray", s=40)

        # Set black background
        ax = plt.gca()
        ax.set_facecolor('xkcd:black')

        plt.xlim(x_min-10, x_max+10)
        plt.ylim(y_min-10, y_max+10)
        plt.xlabel('X')
        plt.ylabel('Y')
        plt.title(f'Firefly sky - time step: {frame_i+1}')
        
        if track_on:
            plt.legend(loc=(1.01, 0.2), ncol=2, title="Male firefly", columnspacing=0.5)
        
    plt.savefig(f'MovieFrames/t_{frame_i:03d}', bbox_inches='tight', dpi=200)
    plt.close()
        

In [ ]:
x_flashes[global_i]

In [ ]:
all_img_paths = np.sort(glob.glob("MovieFrames/*.png"))
all_imgs = np.array([cv2.imread(img) for img in all_img_paths])

def imgs2vid(imgs, outpath, fps=20):
    height, width, layers = imgs[0].shape
    fourcc = cv2.VideoWriter_fourcc("m", "p", "4", "v")
    video = cv2.VideoWriter(outpath, fourcc, fps, (width, height), True)
    
    for img in imgs:
        video.write(img)
        
    cv2.destroyAllWindows()
    video.release()
    
if track_on:
    imgs2vid(all_imgs, 'MovieFrames/FireflySky.mp4')
else:
    imgs2vid(all_imgs, 'MovieFrames/FireflySky_OnlyFlash.mp4')

Ideas for interface:
- Make it a story: Read in videos of track+flashing and only flashing
- Ask user to provide parameters from sliders
- Generate simulation
- Display results

In [ ]:
# widgets:
ticks = Text(description="Ticks:")
# sheep = Text(description="sheep:", margin=5, width="60px")
# wolves = Text(description="wolves:", margin=5, width="60px")
# grass = Text(description="grass/4:", margin=5, width="60px")
canvas = HTML(margin=10)
plot = HTML()
setup_button = Button(description="setup", width="47%", margin=5)
go_button = Button(description="go", width="47%", margin=5)

# Model parameters
num_males_slider = IntSlider(description="Number of male fireflies:", min=1, max=100, margin=5, width="100px")
step_size_slider = IntSlider(description="Step size:", min=1, max=5, margin=5, width="100px")
num_total_steps_slider = IntSlider(description="Number of total steps:", min=1, max=1000, margin=5, width="100px")
turning_angle_slider = IntSlider(description="Width of turning angle:", min=np.pi/10, max=np.pi/2, margin=5, width="100px")
flash_min_slider = IntSlider(description="Min flashing period:", min=1, max=100, margin=5, width="100px")
flash_max_slider = IntSlider(description="Max flashing period:", min=1, max=100, margin=5, width="100px")

# grass_checkbox = Checkbox(description="grass?", margin=5)
# grass_regrowth_time = IntSlider(description="grass_regrowth_time:",
#                                min=0, max=100, margin=5, width="300px")
# sheep_slider = IntSlider(description="initial_number_sheep:", min=0, max=250, margin=5, width="100px")
# wolves_slider = IntSlider(description="initial_number_wolves:", min=0, max=250, margin=5, width="100px")
# sheep_gain_from_food_slider = IntSlider(description="sheep_gain_from_food:", min=0, max=50, margin=5, width="100px")
# wolf_gain_from_food_slider = IntSlider(description="wolf_gain_from_food:", min=0, max=100, margin=5, width="100px")
# sheep_reproduce_slider = FloatSlider(description="sheep_reproduce:", min=0, max=20, margin=5, width="100px")
# wolf_reproduce_slider = FloatSlider(description="wolf_reproduce:", min=0, max=20, margin=5, width="100px")

# layout:
row1 = HBox([setup_button, go_button], background_color="lightblue", width="100%")
row2 = HBox([num_males_slider, step_size_slider], background_color="lightgreen", width="100%")
row3 = HBox([num_total_steps_slider, turning_angle_slider], background_color="lightgreen", width="100%")
row4 = HBox([flash_min_slider, flash_max_slider, background_color="lightgreen", width="100%")
row5 = HBox([plot], width="100%")
widgets = HBox([VBox([row1, row2, row3, row4, row5], width="60%"), 
                VBox([ticks, canvas], background_color="Khaki")], width="100%")

# row5 = HBox([sheep_reproduce_slider, wolf_reproduce_slider], background_color="lightgreen", width="100%")
# row61 = HBox([sheep], background_color="Khaki", width="100%")
# row62 = HBox([wolves], background_color="Khaki", width="100%")
# row63 = HBox([grass], background_color="Khaki", width="100%")

########################################
             
# update:
ticks.value = str(world.ticks)
canvas.value = str(world.draw(15))
plot.value = world.plot(6.0, 3.0)
grass_checkbox.value = world.use_grass
grass_regrowth_time.value = world.grass_regrowth_time
sheep_slider.value = world.initial_number_sheep
wolves_slider.value = world.initial_number_wolves
sheep_gain_from_food_slider.value = Sheep.GAIN_FROM_FOOD
wolf_gain_from_food_slider.value = Wolf.GAIN_FROM_FOOD
sheep_reproduce_slider.value = Sheep.REPRODUCE
wolf_reproduce_slider.value = Wolf.REPRODUCE
stats = world.get_stats()
sheep.value = str(stats[0])
wolves.value = str(stats[1])
grass.value = str(stats[2])
ticks.value = str(world.ticks)

def update(args):
    if args["name"] != "value":
        return
    owner = args["owner"]
    name = args["name"]
    value = args["new"]
    if owner == grass_checkbox:
        world.use_grass = value
    elif owner == grass_regrowth_time:
        world.grass_regrowth_time = value
    elif owner == sheep_slider:
        world.initial_number_sheep = value
    elif owner == wolves_slider:
        world.initial_number_wolves = value
    elif owner == sheep_gain_from_food_slider:
        Sheep.GAIN_FROM_FOOD = value
    elif owner == wolf_gain_from_food_slider:
        Wolf.GAIN_FROM_FOOD = value
    elif owner == sheep_reproduce_slider:
        Sheep.REPRODUCE = value
    elif owner == wolf_reproduce_slider:
        Wolf.REPRODUCE = value

def setup(widgets):
    world.setup()
    canvas.value = str(world.draw(15))
    plot.value = world.plot(6.0, 3.0)
    stats = world.get_stats()
    sheep.value = str(stats[0])
    wolves.value = str(stats[1])
    grass.value = str(stats[2])
    ticks.value = str(world.ticks)

def run(widgets):
    global thread
    if world.state == "stopped":
        go_button.description = "pause"
        w = {"canvas": canvas, "plot": plot, "ticks": ticks, 
             "sheep": sheep, "wolves": wolves, "grass": grass}
        thread = threading.Thread(target=lambda: world.run(w, 6.0, 3.0, 15))
        thread.start()
    else:
        go_button.description = "go"
        world.state = "stopped"
        thread.join()
        clear_output()
    
grass_checkbox.observe(update)
# FIX:
grass_regrowth_time.observe(update)
sheep_slider.observe(update)
wolves_slider.observe(update)
sheep_gain_from_food_slider.observe(update)
wolf_gain_from_food_slider.observe(update)
sheep_reproduce_slider.observe(update)
wolf_reproduce_slider.observe(update)
# END OF FIX

setup_button.on_click(setup)
go_button.on_click(run)

widgets

## Converting original MATLAB/R code with functional programming to Python

### 1. Male firefly correlated random walk

In [ ]:
# Parameters
# Based on these params, a firefly takes 1000 steps and flash every 100 steps (10x)
N = 1000           # Number of steps per trajectory
REALIZATIONS = 20  # Number of fireflies in arena 
v = 1.0            # Velocity aka step size
NreS = 100         # Number of steps bw flashes
sigma02 = np.pi/10 # Width of random walk turning angle distribution. Lower: straighter.

initial_arena_size = N

In [ ]:
# Correlated random walks
np.random.seed(42)

x = np.zeros((REALIZATIONS, N))
y = np.zeros((REALIZATIONS, N))
theta = np.zeros((REALIZATIONS, N))

# Pick random initial positions for all male FF (convert to - to + coordinates)
x[:,0] = initial_arena_size * (np.random.rand(x.shape[0]) - 0.5)
y[:,0] = initial_arena_size * (np.random.rand(y.shape[0]) - 0.5)
theta[:,0] = np.pi * (np.random.rand(theta.shape[0]) - 0.5)

# Loop over each male firefly
for realization_i in range(REALIZATIONS):
    
    # Each FF takes N steps total
    for step_i in range(1, N):
        
        # At each step, it chooses a random turning angle (radians)
        theta[realization_i, step_i] = theta[realization_i, step_i-1] + sigma02 * 2 * (np.random.rand()-0.5)
        
        # Update x and y position according to theta
        x[realization_i, step_i] = x[realization_i, step_i-1] + v * np.cos(theta[realization_i, step_i])
        y[realization_i, step_i] = y[realization_i, step_i-1] + v * np.sin(theta[realization_i, step_i])


In [ ]:
fig = plt.figure(figsize=(8,7))
ax = fig.add_subplot(1, 1, 1)

# Plot line tracks
plt.plot(x.T, y.T, lw=1, zorder=0, label="-")

# Plot flashing patterns
xx = x.T.copy()
yy = y.T.copy()
plt.scatter(xx[::NreS], yy[::NreS], c='orange', s=20)

plt.xlabel('X')
plt.ylabel('Y')

ax = plt.gca()
ax.set_facecolor('xkcd:black')

# plt.legend(loc=(1.01,.1), title=("Male firefly"))
plt.title('Firefly sky')
plt.show()

At first flashes, (assume synchronized flashing?) female picks a single male and attempts to track him until the end of simulation? At each time, firefly makes a guess on most likely male based on flash location, and build a history of that track

## Try to learn widgets

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
import numpy as np

%matplotlib nbagg

Define the function that's changed each time a slider is changes

In [ ]:
def update_plot(amp, phase, freq):
    ax.clear()
    
    units = f"amp={amp} $(psi)$ \nphase={phase} $(secs)$ \nfreq={freq} $(Hz)$"
    y = amp * np.sin(2 * np.pi * (freq * x * phase))
    ax.plot(x, y, label=units)
    ax.legend(loc=1)
    ax.set_xlabel("$sec$")
    ax.set_ylabel("$psi$")
    ax.set_xlim()
    ax.set_ylim()
    plt.show()

In [ ]:
x = np.linspace(0, 2, 1000)
fig, ax = plt.subplots(1, figsize=(10, 4))
plt.suptitle('Sine wave')

# Create sliders
amp = widgets.FloatSlider(min=1, max=10, value=1, description="Amp: ")
phase = widgets.FloatSlider(min=0, max=5, value=0, description="Phase: ")
freq = widgets.FloatSlider(min=1, max=10, value=1, description="Freq: ")

# Link values of widgets to agruments for function
widgets.interactive(update_plot, amp=amp, phase=phase, freq=freq)

display(amp, phase, freq)